<center><u><h1><strong> ROBERTA & BERT </strong></h1></center></u>


<center><img src="https://miro.medium.com/max/2400/0*x3vhaoJdGndvZqmL.png" alt="exemple de texte alternatif" width=300px /></center>

<h2><u>Sommaire :</u></h2>
<ul>
  <li> Importation </li>
  <li> Configuration des données </li>
  <li> Configuration du modèle </li>
  <li> Entrainement du modèle </li>
  <li> Prédictions </li>
  <li> Comparaison des prédictions de différents modèles </li>
  <li> Bibliographie </li>
</ul>


<u> <h2><FONT color="darkred"> I - Importation </h2> </u>

<u><h3><FONT color="darkgreen"> A - Importation des packages </h3></u>

Nous installons les packages manquants transformers et wget. Puis il faut **importer** tous **les packages** pour la suite.

In [3]:
#Installation 
!pip install transformers
!pip install wget

#Import 
import pandas as pd
import wget
import os
import numpy as np
import tensorflow as tf
import torch
import time
import datetime
import random
import plotly.express as px

#FROM
from google.colab import drive
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from sklearn.metrics import matthews_corrcoef
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

     |████████████████████████████████| 1.5MB 7.5MB/s 
     |████████████████████████████████| 2.9MB 36.2MB/s 
     |████████████████████████████████| 890kB 15.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9d6ddbba6342b40f4969be686729bfc1ea843c9455de31f8e97cb65818ecbab7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=1499bf22533fc2c6d62ed41484977134073dcda617638dd08ee6d5dd66531fd8
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


<u><h3><FONT color="darkgreen"> B - Connexion Google Drive </h3></u>

Pour importer les données plus rapidement, nous importons un **google drive** qui comporte les données.

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


<u><h3><FONT color="darkgreen"> C - Importation des données </h3></u>

Nous importons les données d'apprentissage grâce au fichier **train.json** qui comporte notamment les descriptions des métiers et grace au fichier **train_label.csv** qui comporte les labels des métiers correspondant. Nous importons les données test grâce au fichier **test.json** comportant les descriptions test.



In [5]:
DATA_PATH = "./"
train_df = pd.read_json(DATA_PATH+"/train.json")
test_df = pd.read_json(DATA_PATH+"/test.json")
train_label = pd.read_csv(DATA_PATH+"/train_label.csv")

train_df["description_lower"] = [x.lower() for x in train_df.description]
test_df["description_lower"] = [x.lower() for x in test_df.description]

sentences=train_df["description"].values
labels=train_label.Category.values
#print(sentences)
#print(labels)

<u><h3><FONT color="darkgreen"> D - Visualisation des données </h3></u>

Nous visualisons ci-dessous une partie des variables explicatives train.Il y a deux variables explicatives le **genre** et la **description**.

In [6]:
train_df.head()

,Id,description,gender,description_lower
0,0,She is also a Ronald D. Asmus Policy Entrepre...,F,she is also a ronald d. asmus policy entrepre...
1,1,He is a member of the AICPA and WICPA. Brent ...,M,he is a member of the aicpa and wicpa. brent ...
2,2,Dr. Aster has held teaching and research posi...,M,dr. aster has held teaching and research posi...
4,3,He runs a boutique design studio attending cl...,M,he runs a boutique design studio attending cl...
5,4,"He focuses on cloud security, identity and ac...",M,"he focuses on cloud security, identity and ac..."


<u><h3><FONT color="darkgreen"> E - GPU </h3></u>

Nous demandons un GPU dans cette partie. Au préalable il faut aller dans :
<fieldset><center>
 <strong> Modifier - Paramètres du Notebook - Accélérateur matériel => GPU </strong></center>
 </fieldset>
 
  Si aucun est disponible on utilise le CPU à la place.

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch
torch.cuda.empty_cache()
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Il faut prendre le GPU le plus puissant qui est ici le **Tesla P100-PCIE-16GB** pour plus de rapidité. 

<u><h2><FONT color="darkred"> II Préparation des données </h2></u>

Avant de passer aux configurations de nos modèles il est important de configurer nos données d'entrée.

<u><h3><FONT color="darkgreen"> A - Chargement Tokenizer </h3></u>

<p> Il faut choisir un des deux Tokenizer correspondant au modèle que l'on souhaite utiliser par la suite. Il existe plusieurs librairies pour chaque tokenizer.</p>

<h4> 1 - Roberta Tokenizer For Sequence Classification </h4>

Nous utilisons ici **RobertaTokenizer** avec la librairie **"roberta-base"**. Il en existe d'autres consultables sur le site huggingface. Nous aurions pu utiliser roberta large qui est entrainée sur plus de mots seulement pour des problèmes de computationnelle il n'était pas possible de l'exécuter sur Google Colab avec ce jeu de données.

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

<h4> 2 - Bert Tokenizer For Sequence Classification </h4>

Nous utilisons ici **BertTokenizer** avec la librairie **"bert-base-uncased"**. Nous aurions pu utiliser bert large qui est entrainé sur plus de mots seulement pour des problèmes de computationnelle il n'était pas possible de l'exécuter sur Google Colab avec ce jeu de données.

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

<u><h3><FONT color="darkgreen"> B - Tokenization </h3></u>

Nous allons tokenizer les phrases c'est-à-dire réaliser un découpage par mot et dans un second temps affecter des ID à chaque mot. Comme on l'a vu précédemment il existe plusieurs techniques de tokenization.

In [8]:
#Tokenization de toutes les phrases et affectation de l'id pour chaque mots 
input_ids = []
for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,                      
                        add_special_tokens = True, 
                        #max_length = 128,          # Tronquez toutes les phrases.
                        #return_tensors = 'pt',     # Renvoie pytorch tensors
                   )
    
    #Ajout de la phrase encoded à la liste inputs_ids
    input_ids.append(encoded_sent)

#Affichage de la première phrase et de la liste des id
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors


Original:   She is also a Ronald D. Asmus Policy Entrepreneur Fellow with the German Marshall Fund and is a Visiting Fellow at the Centre for International Studies (CIS) at the University of Oxford. This commentary first appeared at Sada, an online journal published by the Carnegie Endowment for International Peace.
Token IDs: [101, 2016, 2003, 2036, 1037, 8923, 1040, 1012, 2004, 7606, 3343, 10670, 3507, 2007, 1996, 2446, 5832, 4636, 1998, 2003, 1037, 5873, 3507, 2012, 1996, 2803, 2005, 2248, 2913, 1006, 20199, 1007, 2012, 1996, 2118, 1997, 4345, 1012, 2023, 8570, 2034, 2596, 2012, 6517, 2050, 1010, 2019, 3784, 3485, 2405, 2011, 1996, 11298, 15108, 2005, 2248, 3521, 1012, 102]


In [9]:
print('La phrase la plus longue possède : ', max([len(sen) for sen in input_ids]), ' caractères' )

La phrase la plus longue possède :  595  caractères


Sur le chunk suivant nous allons choisir la **longueur maximale en caractère des phrases** de l'entrainement. Pour des raisons computationnelles nous choissions **486 caractères** au-delà de cette valeur nous rencontrons des problèmes.

In [10]:
MAX_LEN = 486
print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Remplir nos tokens d'entrée avec la valeur 0.
# "post" indique que nous voulons remplir et tronquer à la fin de la séquence, par opposition au début.

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
print('\Done.')


Padding/truncating all sentences to 486 values...

Padding token: "[PAD]", ID: 0
\Done.


Nous allons créer des **masques d'attention**. Le masque d'attention indique simplement quels tokens sont des mots réels par rapport à ceux qui ne le sont pas.

In [11]:
# Création du "attention masque" pour toutes les phrases.
attention_masks = []
for sent in input_ids:

    # - Si un token ID est 0, alors c'est un padding, définissez le masque sur 0.
    # - Si un token ID est> 0, alors c'est un vrai token, définissez le masque sur 1.

    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Ajout du "attention mask" de la phrase
    attention_masks.append(att_mask)

On réalise un **découpage apprentissage/validation** pour les inputs/labels et pour les masks pour l'entrainement avec comme proportion **90%** pour l'apprentissage et **10%** pour la validation.



In [12]:
# Inputs/Labels
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
# Train/Validation
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

On convertit les inputs/labels et masks en **torch tensors**.

In [13]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)


Dans le chunk ci-dessous on choisit la **taille des batchs**, les valeurs recommandées pour le fine-tuning pour des taches spécifiques de ce type de modèle Bert ou Roberta sont de **16 ou 32**. Nous avons opté pour des batchs de taille 16 car malgré le fait que cette taille augmente le temps d'exécution elle permet de prendre une longueur maximale des phrases en caractères plus grande. Nous allons également créer un **itérateur** pour notre ensemble de données à l'aide de la classe **Torch DataLoader**. Cela permet d'économiser de la mémoire pendant l'entraînement car, contrairement à une boucle for, avec un itérateur, l'ensemble de données n'a pas besoin d'être chargé en mémoire.

In [14]:
# Choix batch size
batch_size = 16

# Créez le DataLoader pour notre ensemble d'apprentissage.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Créez le DataLoader pour notre ensemble de validation.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<u><h2><FONT color="darkred"> III - Configuration du modèle </h2> </u>

Nous allons dans cette partie configurer notre modèle. Il convient de choisir un des deux modèles possible en cohérence avec le tokenizer choisi en amont.

<u><h3><FONT color="darkgreen"> A - Choix du modèle </h3></u>

<h4> 1 - Modèle RobertaForSequenceClassification </h4>

Nous configurons le modèle avec le paramètre **num_labels=28** qui correspond au nombre de labels dans la variable a expliquer qui est de 28 dans notre cas. Notre modèle pré entrainé avec la librairie **roberta-base** comme nous l'avons mentionné précédemment.

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=28,
    output_attentions=False,
    output_hidden_states=False)

model.cuda()

<h4> 2 - Modèle BertForSequenceClassification </h4>

Nous prenons le même **num_labels**. Ici la librairie de pré-traitement est **bert-base-uncased**.

In [15]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 28,    
    output_attentions = False, # Affichage du modèle des attentions poids  
    output_hidden_states = False, # Affichage du modèle de toutes les couches cachés
)
# Indique à pytorch d'executer le modèle sur le GPU
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

<u><h3><FONT color="darkgreen"> B - Visualisation des paramètres du modèles </h3></u>

Nous visualisons certains paramètres de notre modèle notamment sur les Embedding Layer.

In [16]:
params = list(model.named_parameters())
print('The BERT model has {:} different named parameters.\n'.format(len(params)))
print('==== Embedding Layer ====\n')
for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== First Transformer ====\n')
for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print('\n==== Output Layer ====\n')
for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

<u><h3><FONT color="darkgreen"> C - Optimisation AdamW </h3></u>

Dans le chunk suivant nous optimisons les paramètres de notre modèle avec AdamW. Nous choisissons  également le nombre d'epoch.

In [17]:
from transformers import AdamW
# Note: AdawW est une classe venant de librarie d'hugging face (opposé à pytorch)
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - par défaut 5e-5, ici 2e-5
                  eps = 1e-8 # args.adam_epsilon  - par défaut 1e-8.
                )
from transformers import get_linear_schedule_with_warmup

# Nombre d'epoch pour le training (entre 2 et 4)
epochs = 3

# Nombre total d'étapes : nombre de batchs * nombre d'epochs
total_steps = len(train_dataloader) * epochs

# Créez le planificateur de taux d'apprentissage.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

<u><h2><FONT color="darkred"> IV Entrainement du modèle sur le train et validation </h2></u>

Nous allons entrainer notre modèle sur la partie train et verifiée sur la partie validation.

<u><h3><FONT color="darkgreen"> A - Définition de fonction </h3></u>




<h5><strong> flat accuracy </strong></h5>

Nous implémentons une fonction pour calculer la précision de nos prédictions par rapport aux labels.

In [18]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

<h5> <strong>format time</strong> </h5>

Nous allons implémenter une fonction pour calculer le temps d'exécution.

In [19]:
def format_time(elapsed):
    # Arrondir à la seconde la plus proche
    elapsed_rounded = int(round((elapsed)))
    
    # Format hh:mm:ss ce qui correspond à Heure:Minute:Seconde
    return str(datetime.timedelta(seconds=elapsed_rounded))


<u><h3><FONT color="darkgreen"> B - Entrainement </h3></u>

Ci-dessous le chunk d'entrainment et de validation.

In [ ]:
# Initialisation d'un seed pour avoir des résultats reproductibles
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Stockez la perte moyenne après chaque époque afin de pouvoir les tracer.
loss_values = []

training_stats = []

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Apprentissage
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Mesure du temps écoulé pour l'apprentisagge d'une epoch
    t0 = time.time()
    
    # Réinitialisez la loss totale pour cette époque.
    total_loss = 0

    # Model en training mode
    model.train()

    for step, batch in enumerate(train_dataloader):

        # Progrès mis à jour tous les 40 batches.
        if step % 40 == 0 and not step == 0:
            
            # Calculez le temps écoulé en minutes.
            elapsed = format_time(time.time() - t0)
            
            # Affichage de la progression
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))


        # Sortir le batch training de notre dataloader 

        # Au fur et à mesure que nous déballons("unpack") le batch, 
        # nous copierons également chaque tenseur sur le GPU en utilisant la méthode `to`.
        
        # Batchs contiennent trois pytorch tensors :
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Effacez toujours les gradients précédemment calculés avant d'effectuer une backward pass. 
        # PyTorch ne le fait pas automatiquement car l'accumulation des gradients est "pratique lors de l'entrainement des RNN".

        model.zero_grad()  

        # Effectuer une forward pass (évaluer le modèle sur ce batch d'entraînement).
        # Cela renverra la perte/loss (plutôt que la sortie du modèle) car nous avons fourni les `labels`.

        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # L'appel à `model` retourne toujours un tuple, 
        # nous devons donc extraire la valeur de perte/loss du tuple.

        loss = outputs[0]
        
        # Accumuler la perte/loss d'entraînement sur tous les batchs afin de pouvoir calculer la perte/loss moyenne à la fin.
        # «loss» est un Tensor contenant une valeur unique; 
        # la fonction `.item ()` renvoie simplement la valeur Python du tenseur.

        total_loss += loss.item()

        # Effectuez une backward pass pour calculer les gradients.
        loss.backward()

        # Fixez la norme des gradients à 1.0.
        # Ceci permet d'éviter le problème des "exploding gradients".

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Mettez à jour les paramètres et faites un pas en utilisant le gradient calculé.

        # L'optimiseur dicte la "règle de mise à jour" 
        # comment les paramètres sont modifiés en fonction de leurs gradients, du taux d'apprentissage, etc.

        optimizer.step()
        
        # Mis à jour du taux d'apprentissage("learning rate")
        scheduler.step()

    # Calculez la perte moyenne sur les données d'entraînement. 
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Stockez la valeur de perte pour tracer la courbe d'apprentissage.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    # Après chaque epoch de training, mesure de  notre performance sur
    # notre ensemble de validation.

    print("")
    print("Running Validation...")
    t0 = time.time()

    # Mettre le modèle en mode évaluation
    model.eval()

    # Suivre variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Suivre variables 
    total_eval_loss = 0

    for batch in validation_dataloader:
        
        #Ajout batch au GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Sortir le batch training de notre dataloader 
        b_input_ids, b_input_mask, b_labels = batch
        
        # Dire au modèle de ne pas calculer ou stocker les gradients,
        # économiser de la mémoire et accélérer la validation

        with torch.no_grad():        

            # Calculer les logits

            # Cela renverra les logits plutôt que la perte car nous n'avons pas fourni d'étiquettes.

            # token_type_ids est le même que les "segment ids", 
            # qui différencient les phrases 1 et 2 dans les tâches à 2 phrases.

            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
            
        # Prendre les logits sortis par le modèle    
        logits = outputs[0]
        
        # Les "logits" sont les valeurs de sortie avant d'appliquer une fonction d'activation comme le softmax.

        # Déplacer les logits et les labels vers le CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculez la précision  pour ce batch pour les phrases test.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
  
        # Somme pour la précision totale
        eval_accuracy += tmp_eval_accuracy

        # Suivre/Actualiser le nombre de batch
        nb_eval_steps += 1

         
    # Affichage de la précision finale de ce validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
print("")
print("Training complete!")



======== Epoch 1 / 3 ========
Training...
  Batch    40  of  12,218.    Elapsed: 0:00:33.
  Batch    80  of  12,218.    Elapsed: 0:01:07.
  Batch   120  of  12,218.    Elapsed: 0:01:40.
  Batch   160  of  12,218.    Elapsed: 0:02:13.
  Batch   200  of  12,218.    Elapsed: 0:02:46.
  Batch   240  of  12,218.    Elapsed: 0:03:19.
  Batch   280  of  12,218.    Elapsed: 0:03:52.
  Batch   320  of  12,218.    Elapsed: 0:04:25.
  Batch   360  of  12,218.    Elapsed: 0:04:59.
  Batch   400  of  12,218.    Elapsed: 0:05:32.
  Batch   440  of  12,218.    Elapsed: 0:06:05.
  Batch   480  of  12,218.    Elapsed: 0:06:38.
  Batch   520  of  12,218.    Elapsed: 0:07:11.
  Batch   560  of  12,218.    Elapsed: 0:07:44.
  Batch   600  of  12,218.    Elapsed: 0:08:18.
  Batch   640  of  12,218.    Elapsed: 0:08:51.
  Batch   680  of  12,218.    Elapsed: 0:09:24.
  Batch   720  of  12,218.    Elapsed: 0:09:57.
  Batch   760  of  12,218.    Elapsed: 0:10:30.
  Batch   800  of  12,218.    Elapsed: 0:11:0

Nous avons réaliser un graphique pour observer la **perte/loss d'entrainement** du modèle en fonction des epochs.


In [ ]:
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

Il est possible de rajouter certaines variables pour visualiser la présence ou non de **surapprentissage** .

<u><h2><FONT color="darkred"> V Prédictions </h2></u>

Maintenant que nous avons configuré, optimisé et entrainé notre modèle nous allons **prédire les labels** avec le jeu de données **test**.

<u><h3><FONT color="darkgreen"> A - Configuration données test </h3></u>

Tout d'abord il convient de configurer le jeu de données test comme le jeu de données train (Tokenization, Masque, Torch Tensor, Batch Size, DataLoader)

In [ ]:
# Définition de sentences/labels
sentences=test_df["description"].values
labels=train_label[0:54300].Category.values

# Affichage du nombre de phrase
print('Le jeu de données test comporte : {:,}\n'.format(test_df.shape[0]),"phrase")

# Tokenization du jeu de données test
input_ids = []
for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,                      
                        add_special_tokens = True, 
                   )
    
    input_ids.append(encoded_sent)

# Pad_sequences : réglages de la longueur maximale
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Attention masks
attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convertion en torch tensor
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

# Choix du batch size.  
batch_size = 32 

# Créer DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

<u><h3><FONT color="darkgreen"> B - Prédictions du jeu de données Test </h3></u>

Nous allons à l'aide du modèle construire/prédire les **logits** des données test.

In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Model en mode eval
model.eval()

# variables 
predictions , true_labels = [], []
# predictions
for batch in prediction_dataloader:

  # Ajout batch au GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Dire au modèle de ne pas calculer ou stocker les gradients, économiser de la mémoire et accélérer la prédiction 
  with torch.no_grad():
      # calculer les logits de prédiction.
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]

  # Déplacer les logits et labels vers CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Stockage des prédictions et des true labels 
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')



Pour construire le vecteur de prédictions il faut garder le **labels** correspand au **logits maximal** pour chaque observations. Le tableau est un double tableau de taille **1697*32** ici. Le 32 correspond à la taille des batchs.

In [ ]:
vect=[]
for j in range(1696):
  for i in range(32):
    pred_labels = np.argmax(predictions[j][i], axis=0).flatten()
    vect.append(pred_labels[0])
for t in range(28):
  pred_labels = np.argmax(predictions[1696][t], axis=0).flatten()
  vect.append(pred_labels[0])
  
print("Le vecteur possède est:",len(vect))

Nous stockons nos prédictions dans un csv avec deux colonnes : la première correspondant à l'identifiant de l'observation elle s'appelle **Id** et la deuxième correspond aux labels prédit qui a pour nom **Category**.

In [ ]:
test_df["Category"] = vect
baseline_file = test_df[["Id","Category"]]
baseline_file.to_csv("BERTAouf486_epoch2.csv", index=False)

<u><h2><FONT color="darkred"> VI Comparaison des prédictions pour différents modèles </h2></u>

Il est possible de comparer les modèles sur leurs prédictions ce qui permet de voir quels sont les modèles les plus proches et permet de donnée une première idée du taux erreur possible en fonction de la proximité avec d'autre modèles où l'erreur de prédiction est connue.

<u><h3><FONT color="darkgreen"> A - Chargement des csv de prédictions </h3></u>

In [ ]:
DATA_PATH = "./"

# ROBERT

# epoch = 1

roberta480 = pd.read_csv(DATA_PATH+"/ROBERTAouf480.csv")
roberta486 = pd.read_csv(DATA_PATH+"/ROBERTAouf486.csv")
roberta320 = pd.read_csv(DATA_PATH+"/ROBERTAouf320.csv")

# epoch = 2

roberta484E2 = pd.read_csv(DATA_PATH+"/ROBERTAouf484_Epoch2.csv")
robert_2e = pd.read_csv(DATA_PATH+"/ROBERTAouf484_Epoch2.csv")

# BERT

# epoch = 1
bert480 = pd.read_csv(DATA_PATH+"/BERTouf_480.csv")
bert_480 = pd.read_csv(DATA_PATH+"/BERTouf_480.csv")

# epoch = 2
bert_2e = pd.read_csv(DATA_PATH+"/BERTouf486_e2.csv")


<u><h3><FONT color="darkgreen"> B - Tableau du pourcentage de similitudes </h3></u>

In [ ]:
print(sum(roberta480["Category"]==roberta484E2["Category"])/54300)
print(sum(roberta486["Category"]==roberta484E2["Category"])/54300)

print(sum(bert_2e["Category"]==bert_480["Category"])/54300)
print(sum(robert_2e["Category"]==bert_2e["Category"])/54300)
print(sum(robert_2e["Category"]==bert_480["Category"])/54300)

In [ ]:
tot_=sum(roberta480["Category"]==roberta486["Category"])/54300
tot_diff2=sum(roberta320["Category"]==roberta486["Category"])/54300
tot_diff3=sum(bert480["Category"]==roberta486["Category"])/54300
print(tot_)
print(tot_diff2)
print(tot_diff3)

In [ ]:
tot_diff=sum(roberta480["Category"]==roberta320["Category"])/54300
tot_diff3=sum(roberta480["Category"]==bert480["Category"])/54300
print(tot_diff)
print(tot_diff3)

In [ ]:
rob_epoch1=sum(roberta480["Category"]==roberta320["Category"])/54300
rob_epoch2=sum(roberta480["Category"]==bert480["Category"])/54300
rob_epoch3=sum(roberta320["Category"]==bert480["Category"])/54300
print(rob_epoch1)
print(rob_epoch2)
print(rob_epoch3)

<u><h3><FONT color="darkgreen"> C - Coefficients de Matthews </h3></u>

Cette partie est réalisable si seulement si on possède les labels du test. 

In [ ]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [ ]:
matthews_set = []
# Évaluer chaque batch de test en utilisant le coefficient de corrélation de Matthew
print('Calculating Matthews Corr. Coef. for each batch...')
for i in range(len(true_labels)):
  
  # Les prédictions pour ce batch sont un ndarray à 2-colonnes (une colonne pour "0" et une colonne pour "1"). 
  #Choisissez le label avec la valeur la plus élevée et transformez-la en une liste de 0s et 1s.

  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculer et stocker le coef pour ce batch
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)


In [ ]:
matthews_set

In [ ]:
# Combinez les prédictions pour chaque batch en une seule liste de 0s et 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combinez les bonnes labels pour chaque batch en une seule liste.
flat_true_labels = [item for sublist in true_labels for item in sublist]
# Cacluer le MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
print('MCC: %.3f' % mcc)

<u><h2><FONT color="darkred"> VII - Bibliographie </h2></u>

<ul>
  <li> https://towardsdatascience.com/fine-tuning-bert-and-roberta-for-high-accuracy-text-classification-in-pytorch-c9e63cf64646  </li>

  <li>https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1 </li>
  
</ul>